In [39]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import joblib
#from convlstm import ConvLSTM
from tqdm import tqdm
import wandb
#wandb.login(key='')

In [40]:
"""
ConvLSTM architecture. All credits goes to https://github.com/ndrplz/ConvLSTM_pytorch 
"""

class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              #padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):

    """

    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.

    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

Load data

In [41]:
interpolated_weather_df = pd.read_csv('data/region/vietnam/interpolated_weather.csv', index_col=0, parse_dates=True)
interpolated_air_df = pd.read_csv('data/region/vietnam/interpolated_air.csv', index_col=0, parse_dates=True)
#interpolated_weather_df = pd.read_csv('/kaggle/input/air-and-weather/interpolated_weather.csv', index_col=0, parse_dates=True)
#interpolated_air_df = pd.read_csv('/kaggle/input/air-and-weather/interpolated_air.csv', index_col=0, parse_dates=True)
interpolated_air_df.drop(columns='aqi', inplace=True)

In [42]:
interpolated_weather_df["wind_x_component"] = np.cos(interpolated_weather_df["wind_direction_10m"] / (180 / np.pi))
interpolated_weather_df["wind_y_component"] = np.sin(interpolated_weather_df["wind_direction_10m"] / (180 / np.pi))
interpolated_weather_df.drop(columns='wind_direction_10m', inplace=True)

Train, validation, test split 

In [43]:
air_train, air_test_and_val = interpolated_air_df.loc[:'2023-12-31 23:00:00'], interpolated_air_df.loc['2024-01-01 00:00:00':]
weather_train, weather_test_and_val = interpolated_weather_df.loc[:'2023-12-31 23:00:00'], interpolated_weather_df.loc['2024-01-01 00:00:00':]


In [44]:
indices = np.arange(len(air_test_and_val))
half_way = int(len(indices) * 0.5)

air_valid = air_test_and_val.iloc[:half_way]
air_test = air_test_and_val.iloc[half_way:]

weather_valid = weather_test_and_val.iloc[:half_way]
weather_test = weather_test_and_val.iloc[half_way:]

Sort by province and time

In [45]:
air_train = air_train.reset_index().sort_values(by=['province', 'time'])
weather_train = weather_train.reset_index().sort_values(by=['province', 'time'])

air_valid = air_valid.reset_index().sort_values(by=['province', 'time'])
weather_valid = weather_valid.reset_index().sort_values(by=['province', 'time'])

air_test = air_test.reset_index().sort_values(by=['province', 'time'])
weather_test = weather_test.reset_index().sort_values(by=['province', 'time'])


In [17]:
air_train.drop(columns=['province', 'time'], inplace=True)
weather_train.drop(columns=['province', 'time'], inplace=True)

air_valid.drop(columns=['province', 'time'], inplace=True)
weather_valid.drop(columns=['province', 'time'], inplace=True)

air_test.drop(columns=['province', 'time'], inplace=True)
weather_test.drop(columns=['province', 'time'], inplace=True)

In [18]:
class TimeSeries3DDataset(Dataset):
    def __init__(self, target, features, n_provinces, sequence_length=3):
        """
        Args:
            target (Dataframe): Input dataframe
            features (Dataframe): Output dataframe
            n_provinces (int): number of channels
            sequence_length (int, optional): Time window size. Defaults to 3.
        """
        self.target_scaler = preprocessing.StandardScaler()
        self.features_scaler = preprocessing.StandardScaler()
        self.features = self.features_scaler.fit_transform(features.values)
        self.target = self.target_scaler.fit_transform(target.values)

        self.X = torch.tensor(self.features.reshape(n_provinces, len(features)//n_provinces, len(features.columns))).float()
        self.y = torch.tensor(self.target.reshape(n_provinces, len(target)//n_provinces, len(target.columns))).float()

        self.sequence_length = sequence_length
        self.features_length = self.features.shape[-1]
        self.target_length = self.target.shape[-1]


    def __len__(self):
        return self.X.shape[1]

    def _mirror_padding(self, x, sequence_length, padding_needed):
        mirrored_part = torch.flip(x[:, :padding_needed, :], dims=[1])
        padded_x = torch.cat([mirrored_part, x], dim=1)
        return padded_x

    def _get_window(self, X, i):
        """
        Get time window, mirror padding if needed
        """
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = X[:, i_start:(i + 1), :]
        else:
            padding_needed = self.sequence_length - i - 1
            x = self._mirror_padding(X, self.sequence_length, padding_needed)
            x = x[:, :self.sequence_length, :]
        return x

    def __getitem__(self, i):
        x_window = self._get_window(self.X, i)
        x_window = x_window.permute(1, 0, 2).unsqueeze(2)
        return x_window, self.y[:, i, :].flatten()

In [19]:
class ConvLSTMTimeSeries(nn.Module):
    """
    ConvLSTM with Linear Layer as final layer
    """
    def __init__(self, input_dim, hidden_dim, input_width, output_width):
        super().__init__()
        self.conv_lstm = ConvLSTM(
            input_dim = input_dim,
            hidden_dim = hidden_dim,
            kernel_size = (1 , input_width),
            num_layers = len(hidden_dim),
            batch_first = True
            )
        self.linear = nn.Linear(hidden_dim[-1] * input_width, input_dim*output_width)
        self.flatten = nn.Flatten(1, -1)

    def forward(self, X):
        _, last_states = self.conv_lstm(X)
        X = last_states[0][0]
        X = self.flatten(X)
        X = self.linear(X)

        return X

    def predict(self, X, numpy_output=True):
        self.eval()
        with torch.no_grad():
            output = self(X)
        if numpy_output:
            output = output.numpy()
        return output

Bayes Hyperparameter Tuning

In [20]:
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "val_loss",
        "goal": "minimize"
    },
    "parameters": {
        "lr": {
            "values": [1e-3, 1e-4]
        },
        "weight_decay": {
            "values": [0, 1e-5, 1e-3]
        },
        "patience": {
            "values": [3, 5, 10]
        },
        "hidden_dim": {
            "values": [[128], [128, 64], [256, 128, 64], [128, 64, 32]]
        }
    }
}
#sweep_id = wandb.sweep(sweep_config, project="data_science")

In [21]:
n_provinces = 63
window_size = 3

train_dataset = TimeSeries3DDataset(air_train, weather_train, n_provinces, window_size)
valid_dataset = TimeSeries3DDataset(air_valid, weather_valid, n_provinces, window_size)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers = 0, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers = 0, pin_memory=True)

model = ConvLSTMTimeSeries(
    input_dim = n_provinces,
    hidden_dim = [256, 128, 64],
    input_width = train_dataset.features_length,
    output_width = train_dataset.target_length)

Save scaler of training data

In [22]:
joblib.dump(train_dataset.features_scaler, 'models/weather_scaler.pickle')
joblib.dump(train_dataset.target_scaler, 'models/air_scaler.pickle')

['models/air_scaler.pickle']

Training loop

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        model = ConvLSTMTimeSeries(
            input_dim = n_provinces,
            hidden_dim = config.hidden_dim,
            input_width = train_dataset.features_length,
            output_width = train_dataset.target_length
            )

        optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=config.patience)
        criterion = nn.MSELoss()
        num_epochs = 50
        val_patience = 20
        waited_epoch = 0
        best_val_loss = float('inf')

        model.to(device)
        model.train()
        
        for epoch in range(num_epochs):
            total_loss = 0
            print(f"\nEpoch {epoch + 1}/{num_epochs}")
            print(f"Learning rate: {optimizer.param_groups[0]['lr']}")

            train_loader = tqdm(train_dataloader, desc="Training", leave=False)
            for X, y in train_loader:
                X, y = X.to(device), y.to(device)

                optimizer.zero_grad()
                output = model(X)
                loss = criterion(output, y)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                train_loader.set_postfix({"Loss": f"{total_loss / (train_loader.n + 1):.4f}"})
            
            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Training Loss: {avg_train_loss:.4f}")

            model.eval()
            total_val_loss = 0
            valid_loader = tqdm(valid_dataloader, desc="Validation", leave=False)
            with torch.no_grad():
                for X, y in valid_loader:
                    X, y = X.to(device), y.to(device)
                    output = model(X)
                    loss = criterion(output, y)
                    total_val_loss += loss.item()
                    valid_loader.set_postfix({"Val Loss": f"{total_val_loss / (valid_loader.n + 1):.4f}"})

            avg_val_loss = total_val_loss / len(valid_dataloader)
            print(f"Validation Loss: {avg_val_loss:.4f}")

            scheduler.step(avg_val_loss)
            wandb.log({"epoch": epoch, "train_loss": avg_train_loss, "val_loss": avg_val_loss})

            if avg_val_loss < best_val_loss:
                waited_epoch = 0
                best_val_loss = avg_val_loss
                torch.save(model.state_dict(), "conv_lstm.pth")
            else:
                waited_epoch += 1
                if waited_epoch >= val_patience:
                    print("Early stopping triggered.")
                    return

In [24]:
#wandb.agent(sweep_id, train, count=30)

Evaluate model

In [25]:
test_dataset = TimeSeries3DDataset(air_test, weather_test, n_provinces, window_size)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers = 0, pin_memory=True)

In [26]:
best_conv_lstm = ConvLSTMTimeSeries(
    input_dim = n_provinces,
    hidden_dim = [256],
    input_width = train_dataset.features_length,
    output_width = train_dataset.target_length
)
best_conv_lstm.load_state_dict(torch.load("models/conv_lstm.pth", map_location="cpu", weights_only=True))

<All keys matched successfully>

In [27]:
with torch.no_grad():
    outputs = []
    true_values = []
    for X, y in test_dataloader:
        output = best_conv_lstm.predict(X, numpy_output=False).squeeze()
        output = output.view(n_provinces, 1, test_dataset.target_length)
        outputs.append(output)

        y = y.squeeze()
        y = y.view(n_provinces, 1, test_dataset.target_length)
        true_values.append(y)

    stacked_outputs = torch.cat(outputs, dim=1)
    original_outputs = stacked_outputs.view(-1, 6)
    original_outputs = train_dataset.target_scaler.inverse_transform(original_outputs)

    stacked_true = torch.cat(true_values, dim=1)
    original_true = stacked_true.view(-1, 6)
    original_true = train_dataset.target_scaler.inverse_transform(original_true)

In [78]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error

Root Mean Squared Error and Mean Absolute Error

In [79]:
pd.Series(root_mean_squared_error(original_outputs, original_true, multioutput="raw_values"), 
          index=["co", "no2", "o3", "so2", "pm2_5", "pm10"])

co       328.524765
no2        7.565302
o3        21.680334
so2        5.373150
pm2_5     29.756366
pm10      32.302020
dtype: float64

In [80]:
pd.Series(mean_absolute_error(original_outputs, original_true, multioutput="raw_values"), 
          index=["co", "no2", "o3", "so2", "pm2_5", "pm10"])

co       187.566911
no2        4.381024
o3        14.947572
so2        2.730824
pm2_5     16.620971
pm10      18.238303
dtype: float64